In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from lxml import html
from bs4 import BeautifulSoup
import requests
import time

In [11]:
# VERSION 1:
def scrape_boohoo(url):
    try:
        # Set Chrome options for headless mode
        chrome_options = Options()
        chrome_options.add_argument("--headless")
        # Initialize the WebDriver
        driver = webdriver.Chrome(options=chrome_options)
        driver.get(url)
        # Get the page source after interactions
        page_source = driver.page_source
        # Close the WebDriver once done to avoid multiple instances
        driver.quit()
        # Parse the page source with Beautiful Soup
        soup = BeautifulSoup(page_source, "html.parser")
        # Extract data using Beautiful Soup methods
        title_element = soup.find("h1", class_="b-product_details-name")
        if title_element:
            title_element = title_element.text.strip()
        data_element = soup.find("p", class_="b-product_details-composition")
        if data_element:
            item = [(title_element, data_element.text)]

            return parse_boohoo_data(item)
        else:
            return "Data element not found on the page."
    except Exception as e:
        return f"An error occurred: {str(e)}"

# URL of the webpage you want to scrape
urls = [
    "https://us.boohoo.com/lapel-crop-blazer/DZZ52779.html?color=123&_gl=1*m20h85*_up*MQ..*_ga*MjAyMzc4NTAyNi4xNzEzMzk2ODY5*_ga_CKX55DLD7G*MTcxMzM5Njg2OC4xLjEuMTcxMzM5Njk1My4wLjAuMA..",
    "https://us.boohoo.com/linen-balloon-sleeve-tie-back-crop-top/GZZ00218-394-18.html?gad_source=1&gclid=Cj0KCQjw0MexBhD3ARIsAEI3WHJaw4iJ5Vq5ZUcUd88vMWpeXn2PRZJlQteKDZ1KrHrfemJCBy6fo70aAr-2EALw_wcB&gclsrc=aw.ds",
    "https://us.boohoo.com/essentials-linen-look-beach-trousers/SZZ00119.html?color=105&_gl=1*cunthv*_up*MQ..*_ga*MTM5NDQ5NDYyMi4xNzE0NjEzODY3*_ga_CKX55DLD7G*MTcxNDYxMzg2Ni4xLjAuMTcxNDYxMzg2Ni4wLjAuMA..&gclid=Cj0KCQjw0MexBhD3ARIsAEI3WHJaw4iJ5Vq5ZUcUd88vMWpeXn2PRZJlQteKDZ1KrHrfemJCBy6fo70aAr-2EALw_wcB&gclsrc=aw.ds",
    "https://us.boohoo.com/chiffon-cowl-neck-maxi-dress/GZZ83250.html?_gl=1*1tots0l*_up*MQ..*_ga*MTM5NDQ5NDYyMi4xNzE0NjEzODY3*_ga_CKX55DLD7G*MTcxNDYxMzg2Ni4xLjEuMTcxNDYxMzk1Mi4wLjAuMA..&gclid=Cj0KCQjw0MexBhD3ARIsAEI3WHJaw4iJ5Vq5ZUcUd88vMWpeXn2PRZJlQteKDZ1KrHrfemJCBy6fo70aAr-2EALw_wcB&gclsrc=aw.ds"
]

scraped_data = []

for url in urls:
    # Call the function to scrape the website
    data = scrape_boohoo(url)
    scraped_data.append(data)
    print(data)

[('Lapel Crop Blazer', "95% Polyester 5% Elastane. Flat Measurement Not Worn: Length To Hem 43cm/17\\'. Machine Washable. Model Wears US size 6.")]
[{'item': 'Lapel Crop Blazer', 'polyester': 95, 'elastane': 5}]
[('Linen Balloon Sleeve Tie Back Crop Top', '95% POLYESTER 5% ELASTANE. MACHINE WASHABLE. MODEL WEARS A US size 6.')]
[{'item': 'Linen Balloon Sleeve Tie Back Crop Top', 'polyester': 95, 'elastane': 5}]
[('Essentials Linen Look Beach Pants', '100% polyester. model wears size M')]
[{'item': 'Essentials Linen Look Beach Pants', 'polyester': 100}]
[('Chiffon Cowl Neck Maxi Dress', '100% Polyester. Machine Wash. Model Wears UK size 10')]
[{'item': 'Chiffon Cowl Neck Maxi Dress', 'polyester': 100}]


In [13]:
# VERSION 2: With dictionary output!
import re

def parse_boohoo_data(scraped_data):
    #print(scraped_data)
    product_dicts = []
    # Define a regular expression to capture percentages and material names
    pattern = re.compile(r'(\d+)%\s+([A-Za-z]+)')

    for item_name, description in scraped_data:
        # Find all matches of the material percentages in the description
        matches = pattern.findall(description)
        # Create a dictionary for each item
        item_dict = {'item': item_name}
        # Populate the dictionary with material compositions
        for percentage, material in matches:
            item_dict[material.lower()] = int(percentage)
        product_dicts.append(item_dict)

    return product_dicts

# # Example data as returned by the scrape_boohoo function
# scraped_data = [
#     ('Lapel Crop Blazer', "95% Polyester 5% Elastane. Flat Measurement Not Worn: Length To Hem 43cm/17'. Machine Washable. Model Wears US size 6."),
#     ('Linen Balloon Sleeve Tie Back Crop Top', '95% POLYESTER 5% ELASTANE. MACHINE WASHABLE. MODEL WEARS A US size 6.'),
#     ('Essentials Linen Look Beach Pants', '100% polyester. model wears size M'),
#     ('Chiffon Cowl Neck Maxi Dress', '100% Polyester. Machine Wash. Model Wears UK size 10')
# ]


# Processing the data
# parsed_data = parse_boohoo_data(scraped_data)
# print(parsed_data)

# VERSION 1:
def scrape_boohoo(url):
    try:
        chrome_options = Options()
        chrome_options.add_argument("--headless")

        driver = webdriver.Chrome(options=chrome_options)

        driver.get(url)

        page_source = driver.page_source

        driver.quit()

        soup = BeautifulSoup(page_source, "html.parser")

        title_element = soup.find("h1", class_="b-product_details-name")
        if title_element:
            title_element = title_element.text.strip()
        data_element = soup.find("p", class_="b-product_details-composition")
        if data_element:
            item = [(title_element, data_element.text)]

            return parse_boohoo_data(item)
        else:
            return "Data element not found on the page."
    except Exception as e:
        return f"An error occurred: {str(e)}"

# URL of the webpage you want to scrape
urls = [
    "https://us.boohoo.com/lapel-crop-blazer/DZZ52779.html?color=123&_gl=1*m20h85*_up*MQ..*_ga*MjAyMzc4NTAyNi4xNzEzMzk2ODY5*_ga_CKX55DLD7G*MTcxMzM5Njg2OC4xLjEuMTcxMzM5Njk1My4wLjAuMA..",
    "https://us.boohoo.com/linen-balloon-sleeve-tie-back-crop-top/GZZ00218-394-18.html?gad_source=1&gclid=Cj0KCQjw0MexBhD3ARIsAEI3WHJaw4iJ5Vq5ZUcUd88vMWpeXn2PRZJlQteKDZ1KrHrfemJCBy6fo70aAr-2EALw_wcB&gclsrc=aw.ds",
    "https://us.boohoo.com/essentials-linen-look-beach-trousers/SZZ00119.html?color=105&_gl=1*cunthv*_up*MQ..*_ga*MTM5NDQ5NDYyMi4xNzE0NjEzODY3*_ga_CKX55DLD7G*MTcxNDYxMzg2Ni4xLjAuMTcxNDYxMzg2Ni4wLjAuMA..&gclid=Cj0KCQjw0MexBhD3ARIsAEI3WHJaw4iJ5Vq5ZUcUd88vMWpeXn2PRZJlQteKDZ1KrHrfemJCBy6fo70aAr-2EALw_wcB&gclsrc=aw.ds",
    "https://us.boohoo.com/chiffon-cowl-neck-maxi-dress/GZZ83250.html?_gl=1*1tots0l*_up*MQ..*_ga*MTM5NDQ5NDYyMi4xNzE0NjEzODY3*_ga_CKX55DLD7G*MTcxNDYxMzg2Ni4xLjEuMTcxNDYxMzk1Mi4wLjAuMA..&gclid=Cj0KCQjw0MexBhD3ARIsAEI3WHJaw4iJ5Vq5ZUcUd88vMWpeXn2PRZJlQteKDZ1KrHrfemJCBy6fo70aAr-2EALw_wcB&gclsrc=aw.ds"
]

scraped_data = []

for url in urls:
    # Call the function to scrape the website
    data = scrape_boohoo(url)
    scraped_data.append(data)
    print(data)

[{'item': 'Lapel Crop Blazer', 'polyester': 95, 'elastane': 5}]
[{'item': 'Linen Balloon Sleeve Tie Back Crop Top', 'polyester': 95, 'elastane': 5}]
[{'item': 'Essentials Linen Look Beach Pants', 'polyester': 100}]
[{'item': 'Chiffon Cowl Neck Maxi Dress', 'polyester': 100}]
